In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

## **Étape 1 : Exploration et Premier Diagnostic** **texte en gras**

In [44]:
file1 = pd.read_csv("/content/drive/MyDrive/GENAI/Week3/Hackathon/MoviesOnStreamingPlatforms.csv")
df1 = pd.DataFrame(file1)
df1.head()

,Unnamed: 0,ID,Title,Year,Age,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Type
0,0,1,The Irishman,2019,18+,98/100,1,0,0,0,0
1,1,2,Dangal,2016,7+,97/100,1,0,0,0,0
2,2,3,David Attenborough: A Life on Our Planet,2020,7+,95/100,1,0,0,0,0
3,3,4,Lagaan: Once Upon a Time in India,2001,7+,94/100,1,0,0,0,0
4,4,5,Roma,2018,18+,94/100,1,0,0,0,0


In [45]:
df1.drop(columns=['Unnamed: 0'], inplace=True)

In [46]:
df1["Type"] = "Movies"

In [47]:
df1.drop_duplicates(inplace=True)

In [48]:
file2 = pd.read_csv("/content/drive/MyDrive/GENAI/Week3/Hackathon/tv_shows.csv")
df2 = pd.DataFrame(file2)
df2.head()

,Unnamed: 0,ID,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Type
0,0,1,Breaking Bad,2008,18+,9.4/10,100/100,1,0,0,0,1
1,1,2,Stranger Things,2016,16+,8.7/10,96/100,1,0,0,0,1
2,2,3,Attack on Titan,2013,18+,9.0/10,95/100,1,1,0,0,1
3,3,4,Better Call Saul,2015,18+,8.8/10,94/100,1,0,0,0,1
4,4,5,Dark,2017,16+,8.8/10,93/100,1,0,0,0,1


In [49]:
df2["Type"] = "TV Shows"

In [50]:
df2.drop(columns=['Unnamed: 0', 'IMDb'], inplace=True)

In [51]:
df2.drop_duplicates(inplace=True)

In [38]:
df3 = pd.concat([df1, df2], axis=0)
df3.head()

,ID,Title,Year,Age,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Type
0,1,The Irishman,2019,18+,98/100,1,0,0,0,Movies
1,2,Dangal,2016,7+,97/100,1,0,0,0,Movies
2,3,David Attenborough: A Life on Our Planet,2020,7+,95/100,1,0,0,0,Movies
3,4,Lagaan: Once Upon a Time in India,2001,7+,94/100,1,0,0,0,Movies
4,5,Roma,2018,18+,94/100,1,0,0,0,Movies


In [41]:
df3.shape

(14883, 10)

In [57]:
import requests
from tqdm import tqdm

tqdm.pandas()

API_KEY = "f0e52ba803b4b1cc7e190ec4c432c706"

mapping = requests.get(
    f"https://api.themoviedb.org/3/genre/movie/list?api_key={API_KEY}&language=en-US"
).json()['genres']
id2name = {g['id']: g['name'] for g in mapping}

def fetch_tmdb_genres(title):
    res = requests.get(
        f"https://api.themoviedb.org/3/search/movie?api_key={API_KEY}&query={title}"
    ).json()
    results = res.get('results') or []
    if results:
        genre_ids = results[0].get('genre_ids', [])
        return [id2name[gid] for gid in genre_ids if gid in id2name]
    return []

df3['GenresList'] = df3['Title'].progress_apply(fetch_tmdb_genres)

100%|██████████| 14883/14883 [20:34<00:00, 12.06it/s]


In [62]:
df3.rename(columns={"GenresList" : "Genre"}, inplace=True)

In [63]:
df3.head()

,ID,Title,Year,Age,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Type,Genre
0,1,The Irishman,2019,18+,98/100,1,0,0,0,Movies,"[Crime, Drama, History]"
1,2,Dangal,2016,7+,97/100,1,0,0,0,Movies,"[Drama, Family, Comedy]"
2,3,David Attenborough: A Life on Our Planet,2020,7+,95/100,1,0,0,0,Movies,"[Documentary, Drama]"
3,4,Lagaan: Once Upon a Time in India,2001,7+,94/100,1,0,0,0,Movies,"[Adventure, Drama, History]"
4,5,Roma,2018,18+,94/100,1,0,0,0,Movies,"[Drama, History, Romance, Mystery]"


In [59]:
df3.shape

(14883, 11)

In [65]:
df3.to_csv("/content/drive/MyDrive/GENAI/Week3/Hackathon/df3.csv")

In [66]:
df = df3.copy()